# 06 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to `Vertex AI` as an `Endpoint` for online prediction serving. The notebook covers the following steps:
1. Run the test steps locally.
2. Execute the model deployment `CI/CD` steps using `Cloud Build`.



## Setup

### Import libraries

In [1]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

### Setup Google Cloud project

In [2]:
PROJECT = 'aiops-industrialization' # Change to your project id.
REGION = 'us-central1' # Change to your region.

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]

print("Project ID:", PROJECT)
print("Region:", REGION)

Project ID: aiops-industrialization
Region: us-central1


### Set configurations

In [3]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

## 1. Run CI/CD steps locally

In [4]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

In [5]:
!py.test src/tests/model_deployment_tests.py::test_model_artifact -s

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.3.0
collecting ... 2021-09-01 11:08:24.665407: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
collected 1 item                                                               

src/tests/model_deployment_tests.py 2021-09-01 11:08:28.592009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-09-01 11:08:28.592064: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-01 11:08:28.592118: I tensorflow/stream_executor/cuda/c

### Run create endpoint

In [6]:
!python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

INFO:google.cloud.bigquery.opentelemetry_tracing:This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.
INFO:root:Creating endpoint chicago-taxi-tips-classifier
INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/175728527123/locations/us-central1/endpoints/5750375444524302336/operations/1544401245286957056
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/175728527123/locations/us-central1/endpoints/5750375444524302336
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/175728527123/locations/us-central1/endpoints/5750375444524302336')
INFO:root:Endpoint is ready.
INFO:root:name: "projects/175728527123/locations/us-central1/endpoints/57503754445

### Run deploy model

In [7]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

INFO:google.cloud.bigquery.opentelemetry_tracing:This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.
INFO:root:serving resources: {'traffic_percentage': 100, 'machine_type': 'n1-standard-2', 'min_replica_count': 1, 'max_replica_count': 1, 'accelerator_type': None, 'accelerator_count': None}
INFO:root:Deploying model chicago-taxi-tips-classifier-v01 to endpoint chicago-taxi-tips-classifier
INFO:google.cloud.aiplatform.models:Deploying Model projects/175728527123/locations/us-central1/models/8287529311943000064 to Endpoint : projects/175728527123/locations/us-central1/endpoints/5750375444524302336
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/175728527123/locations/us-central1/endpoints/5750375444524302336/operations/2584732759209541632
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: pr

### Run model endpoint testing

In [8]:
!py.test src/tests/model_deployment_tests.py::test_model_endpoint

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.3.0
collected 1 item                                                               

src/tests/model_deployment_tests.py .                                    [100%]

=============================== warnings summary ===============================
../../../opt/conda/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:22
  /opt/conda/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
    import imp

-- Docs: https://docs.pytest.org/en/stable/warnings.html
========================= 1 passed, 1 warning in 4.09s =========================


## 2. Execute the Model Deployment CI/CD routine in Cloud Build

The CI/CD routine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in Vertex AI if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying model will be executed.

In [9]:
!echo $CICD_IMAGE_URI

gcr.io/aiops-industrialization/cicd:latest


In [10]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

Creating temporary tarball archive of 9 file(s) totalling 17.4 KiB before compression.
Uploading tarball of [build/.] to [gs://aiops-industrialization_cloudbuild/source/1630495486.388614-3056dd2fa541460dae0270f32a7a53b5.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/aiops-industrialization/locations/global/builds/8a79c0d6-a524-46bf-bdbe-6774da520db9].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/8a79c0d6-a524-46bf-bdbe-6774da520db9?project=175728527123].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8a79c0d6-a524-46bf-bdbe-6774da520db9"

FETCHSOURCE
Fetching storage object: gs://aiops-industrialization_cloudbuild/source/1630495486.388614-3056dd2fa541460dae0270f32a7a53b5.tgz#1630495488276621
Copying gs://aiops-industrialization_cloudbuild/source/1630495486.388614-3056dd2fa541460dae0270f32a7a53b5.tgz#1630495488276621...
/ [1 files][  3.0 KiB/  3.0 KiB]                                            

### Run CI/CD from model deployment using Cloud Build

In [11]:
REPO_URL = "https://github.com/ksalama/ucaip-labs.git" # Change to your github repo.
BRANCH = "main" 

In [12]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

_REPO_URL=https://github.com/ksalama/ucaip-labs.git,_BRANCH=main,_CICD_IMAGE_URI=gcr.io/aiops-industrialization/cicd:latest,_PROJECT=aiops-industrialization,_REGION=us-central1,_MODEL_DISPLAY_NAME=chicago-taxi-tips-classifier-v01,_ENDPOINT_DISPLAY_NAME=chicago-taxi-tips-classifier,


In [15]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m

Created [https://cloudbuild.googleapis.com/v1/projects/aiops-industrialization/locations/global/builds/5d0e6cdb-32c4-49eb-aeb8-88f7a9e58137].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/5d0e6cdb-32c4-49eb-aeb8-88f7a9e58137?project=175728527123].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "5d0e6cdb-32c4-49eb-aeb8-88f7a9e58137"

FETCHSOURCE
BUILD
Starting Step #0 - "Clone Repository"
Step #0 - "Clone Repository": Already have image (with digest): gcr.io/cloud-builders/git
Step #0 - "Clone Repository": 
Step #0 - "Clone Repository":                    ***** NOTICE *****
Step #0 - "Clone Repository": 
Step #0 - "Clone Repository": Official `cloud-sdk` images, including multiple tagged versions across multiple
Step #0 - "Clone Repository": platforms, can be found at
Step #0 - "Clone Repository": https://github.com/GoogleCloudPlatform/cloud-sdk-docker and may be more suitable
Step #0 - "Clone Repository": for